# **HOW TO TRAIN NEW MODEL (En to Vi)**



Phiên bản V1 nhằm chạy thử model TED anh - việt với các đặc điểm:

1 - Chạy  7 epoch

2 - Thực hiện trên máy local

3 - Chạy với tập train theo dạng (s1|||tar_best))


## Data Pre-processing

In [ ]:
# # Download and prepare the data
!bash clc_fairseq/examples/translation/prepare-en-vi-iwslt.sh


Cloning Moses github repository (for tokenization scripts)...
Cloning into 'mosesdecoder'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 147592 (delta 5), reused 11 (delta 3), pack-reused 147572
Receiving objects: 100% (147592/147592), 129.76 MiB | 7.84 MiB/s, done.
Resolving deltas: 100% (114030/114030), done.
Checking out files: 100% (3470/3470), done.
Cloning Subword NMT repository (for BPE pre-processing)...
Cloning into 'subword-nmt'...
remote: Enumerating objects: 576, done.
remote: Total 576 (delta 0), reused 0 (delta 0), pack-reused 576
Receiving objects: 100% (576/576), 233.12 KiB | 3.33 MiB/s, done.
Resolving deltas: 100% (349/349), done.
--2020-11-21 06:37:56--  https://github.com/hoangtrungchinh/clc_data/raw/master/en_vi_iwslt.tar.gz
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sen

# Training

In [1]:
!mkdir -p "ed-iwslt-checkpoints/fconv_wmt_en_vi"
!CUDA_VISIBLE_DEVICES=0 fairseq-train "ed-iwslt-data-bin/tokenized.en-vi" \
    --arch transformer --share-decoder-input-output-embed \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
    --lr 5e-4 --lr-scheduler inverse_sqrt --warmup-updates 400 \
    --dropout 0.3 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-tokens 4096 \
    --eval-bleu \
    --eval-bleu-args '{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}' \
    --eval-bleu-detok moses \
    --eval-bleu-remove-bpe \
    --eval-bleu-print-samples \
    --best-checkpoint-metric bleu --maximize-best-checkpoint-metric \
    --save-dir "ed-iwslt-checkpoints/fconv_wmt_en_vi" \
    --max-epoch 7


tion | example reference: nhưng cũng có một hạn chế.

epoch 001 | valid on 'valid' subset:   2%|▏      | 1/46 [00:25<19:11, 25.59s/it]2020-12-08 13:07:17 | INFO | fairseq.tasks.translation | example hypothesis: và chúng ta có thể làm việc.
2020-12-08 13:07:17 | INFO | fairseq.tasks.translation | example reference: bé ấy được sáu tháng tuổi.

epoch 001 | valid on 'valid' subset:   4%|▎      | 2/46 [00:44<17:14, 23.51s/it]2020-12-08 13:07:35 | INFO | fairseq.tasks.translation | example hypothesis: và chúng ta có thể làm việc.
2020-12-08 13:07:35 | INFO | fairseq.tasks.translation | example reference: mỗi ly chứa một loại dầu thơm khác nhau.

epoch 001 | valid on 'valid' subset:   7%|▍      | 3/46 [01:01<15:36, 21.78s/it]2020-12-08 13:07:50 | INFO | fairseq.tasks.translation | example hypothesis: và chúng ta có thể làm việc.
2020-12-08 13:07:50 | INFO | fairseq.tasks.translation | example reference: các bạn chắc hẳn đã từng nghe về khái niệm big data.

epoch 001 | valid on 'valid' subset:

In [1]:
!pwd

/home/chinh/clc_fairseq


In [2]:
# Evaluate
!mkdir -p ed-iwslt-data-bin/fconv_wmt_en_vi
!fairseq-generate \
    ed-iwslt-data-bin/tokenized.en-vi/ \
    --path ed-iwslt-checkpoints/fconv_wmt_en_vi/checkpoint_best.pt \
    --beam 5 --remove-bpe | tee ed-iwslt-data-bin/fconv_wmt_en_vi/gen.out

cal ideas may be needed , and i think that we may need one or two ideas that initially seem crazy before we can come to grips with consciousness scientifically .
T-772	đối mặt với một thứ dị thường như thế này , sẽ cần những ý tường cấp tiến và thôi nghĩ rằng chúng ta sẽ cần một hoặc hai ý tướng ban đầu , nghe có vẻ điên khùng trước khi chúng ta hiểu biết một cách khoa học .
H-772	-2.3584330081939697	và chúng ta có thể làm việc .
D-772	-2.3584330081939697	và chúng ta có thể làm việc .
P-772	-2.3632 -4.3767 -0.9269 -2.3555 -0.7019 -3.8860 -2.2792 -4.1781 -0.1584
S-19	it acts to maximize future freedom of action , or keep options open , with some strength t , with the diversity of possible accessible futures , s , up to some future time horizon , tau .
T-19	nó hành động để tối đa hóa tự do hành động tương lai , hoặc giữ cho các lựa chọn được để ngỏ , với một cường độ t , với sự đa dạng của các tương lai có thể đạt tới , s trong một ngưỡng thời gian tương lai nào đó , tau .
H-19	-2.358432

Compute BLEU score


In [3]:
!grep ^H ed-iwslt-data-bin/fconv_wmt_en_vi/gen.out | cut -f3- > ed-iwslt-data-bin/fconv_wmt_en_vi/gen.out.sys
!grep ^T ed-iwslt-data-bin/fconv_wmt_en_vi/gen.out | cut -f2- > ed-iwslt-data-bin/fconv_wmt_en_vi/gen.out.ref
!fairseq-score --sys ed-iwslt-data-bin/fconv_wmt_en_vi/gen.out.sys --ref ed-iwslt-data-bin/fconv_wmt_en_vi/gen.out.ref


Namespace(ignore_case=False, order=4, ref='ed-iwslt-data-bin/fconv_wmt_en_vi/gen.out.ref', sacrebleu=False, sentence_bleu=False, sys='ed-iwslt-data-bin/fconv_wmt_en_vi/gen.out.sys')
BLEU4 = 0.18, 25.9/3.7/0.5/0.1 (BP=0.126, ratio=0.325, syslen=8640, reflen=26566)


BLEU4 = 0.18, quá tệ

In [2]:
from fairseq.models.transformer import TransformerModel
model = TransformerModel.from_pretrained(
  'iwslt-checkpoints-v2/fconv_wmt_en_vi',
  checkpoint_file='checkpoint_best.pt',
  bpe='subword_nmt',
  bpe_codes = 'iwslt-tokenized.en-vi/code'
)
print(model.translate('I am going to school'))

cô ấy đang làm việc ở trường học .
